In [144]:
import os
import sys
import pandas as pd
import csv
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_folder = '../data'
data_files = os.listdir(data_folder)

In [6]:
for file in data_files:
    print("for year "+file[-8:-4]+" : " + file)

for year 2014 : 22306_24.65_73.75_2014.csv
for year 2000 : 22306_24.65_73.75_2000.csv
for year 2012 : 22306_24.65_73.75_2012.csv
for year 2013 : 22306_24.65_73.75_2013.csv
for year 2008 : 22306_24.65_73.75_2008.csv
for year 2005 : 22306_24.65_73.75_2005.csv
for year 2007 : 22306_24.65_73.75_2007.csv
for year 2006 : 22306_24.65_73.75_2006.csv
for year 2009 : 22306_24.65_73.75_2009.csv
for year 2001 : 22306_24.65_73.75_2001.csv
for year 2011 : 22306_24.65_73.75_2011.csv
for year 2004 : 22306_24.65_73.75_2004.csv
for year 2010 : 22306_24.65_73.75_2010.csv
for year 2003 : 22306_24.65_73.75_2003.csv
for year 2002 : 22306_24.65_73.75_2002.csv


In [120]:
year_2014 = pd.read_csv(data_folder+'/'+data_files[0])
year_2014

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Wind Direction Units,Wind Speed,Snow Depth Units,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Version
0,SUNY,22306,-,-,India,24.65,73.75,5.5,0,5.5,...,Degrees,m/s,m,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,unknown
1,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,1,0,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1,1,1,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1,1,2,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2014,1,1,3,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,1,1,4,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2014,1,1,5,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2014,1,1,6,30,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,1,1,7,30,2,0,2,9,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
new_columns = year_2014.iloc[1]
year = year_2014[2:]
year.columns =new_columns
year = year.loc[:, year.columns.notnull()]

# for counter,i in enumerate(new_columns):
#     print(i)


year = year.drop(columns=['Year','Month','Day','Minute','Hour','DHI','DNI','Clearsky DHI','Clearsky DNI','Fill Flag'])

In [125]:
year.head()

1,GHI,Clearsky GHI,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Snow Depth,Wind Direction,Wind Speed
2,0,0,4,12.751339552289267,957.9305419921875,58.41139916942352,177.55742015012578,1.3668830583200968,0.0,73.88963317871094,4.430836200714111
3,0,0,6,12.196895002047142,957.7622680664062,67.29592773800267,168.03725969433503,1.4700043477445826,0.0,75.10269927978516,4.453418731689453
4,0,0,7,11.806139366268553,957.8780517578125,72.56749007380043,154.41832139803458,1.5719694344243218,0.0,74.26346588134766,4.3355021476745605
5,0,0,7,11.510595990320212,957.6235961914062,75.29933981073673,140.83665499983437,1.6669825069938207,0.0,72.37870025634766,4.2338972091674805
6,0,0,7,11.306806960929293,957.932861328125,76.7992879237338,127.37552025553532,1.7609774282137391,0.0,70.31179809570312,4.197237968444824


In [145]:
x = np.arange(year.shape[0])[0:100]
y = year['GHI'][0:100]

In [142]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='dkumar', api_key='9LpQuTCd1nsEtNQ8Yfzb')

In [141]:
trace = go.Scatter(
    x = x,
    y = y
)

data = [trace]

py.iplot(data, filename='basic-line')

/home/primus/durgeshapi/ensemble/env/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [146]:
x = np.arange(year.shape[0])
y = year['GHI']
trace = go.Scatter(
    x = x,
    y = y
)

data = [trace]

py.iplot(data, filename='basic-line')